## 问题定义

对待求解问题的数学描述如下：
![Equation](./resource/equation.png)

其中，$u$为待求解物理量，$x$为空间坐标，$t$为时间，$c=1$为波速

并且对于上述求解问题，解析解为：$u=\sin(x)(\sin(t) + \cos(t))$

### 求解目标

给定坐标$(x,t)$求解结果（$u$）

## Causal Training

对于长时序问题，求解结果显然具有因果性。但是一般的训练方法忽略了这一特点，导致容易收敛到错误解。

为解决这个问题提出了Causal Training。这一方法的逻辑很简单：强制要求对于任意时刻$T$，只有当$t<T$的训练足够好时才训练该时刻。

参考论文：Wang, Sifan, Sankaran, Shyam, and Perdikaris, Paris. Respecting causality is all you need for training physics-informed neural networks. arXiv preprint arXiv:2203.07404, 2022.

具体实践上：
1. 将整个时间分为N个阶段
2. 总损失是所有阶段的加权和
3. 每一个节点的权重是前置阶段**损失**的负幂指数（即前置阶段损失越大，后面阶段的损失权重越小，从而阻止后面阶段的训练）

![Causal Training](./resource/causal_training.png)


## 求解

In [1]:
import os
import warnings

# optional
# set appropriate GPU in case of multi-GPU machine
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"

In [2]:
# 必要的符号运算
from sympy import Symbol, Eq, Abs, Function, Number, sin
import numpy as np

import modulus.sym

# 超参数
from modulus.sym.hydra import to_yaml
from modulus.sym.hydra import to_absolute_path, instantiate_arch, ModulusConfig
from modulus.sym.hydra.utils import compose

# 求解器
from modulus.sym.solver import Solver

# loss
from modulus.sym.loss.loss import CausalLossNorm

# domain
from modulus.sym.domain import Domain

# 几何物体
from modulus.sym.geometry.primitives_1d import Line1D
from modulus.sym.geometry.parameterization import OrderedParameterization

# 约束
from modulus.sym.domain.constraint import (
    PointwiseBoundaryConstraint,
    PointwiseInteriorConstraint,
)

# validator
from modulus.sym.domain.validator import PointwiseValidator

# inferencer
from modulus.sym.domain.inferencer import PointwiseInferencer
from modulus.sym.key import Key

# Equation
# 导入抽象基类
from modulus.sym.eq.pde import PDE

# post process
from modulus.sym.utils.io import (
    csv_to_dict,
    ValidatorPlotter,
    InferencerPlotter,
)
import matplotlib.pyplot as plt

In [3]:
cfg = compose(config_path="conf", config_name="config")
cfg.network_dir = 'outputs'    # Set the network directory for checkpoints
print(to_yaml(cfg))

/usr/local/lib/python3.10/dist-packages/modulus/sym/hydra/utils.py:148: UserWarning: 
The version_base parameter is not specified.
Please specify a compatability version level, or None.
Will assume defaults for version 1.1
  hydra.initialize(


training:
  max_steps: 80000
  grad_agg_freq: 1
  rec_results_freq: 1000
  rec_validation_freq: ${training.rec_results_freq}
  rec_inference_freq: ${training.rec_results_freq}
  rec_monitor_freq: ${training.rec_results_freq}
  rec_constraint_freq: ${training.rec_results_freq}
  save_network_freq: 1000
  print_stats_freq: 100
  summary_freq: 1000
  amp: false
  amp_dtype: float16
  ntk:
    use_ntk: false
    save_name: null
    run_freq: 1000
graph:
  func_arch: false
  func_arch_allow_partial_hessian: true
stop_criterion:
  metric: null
  min_delta: null
  patience: 50000
  mode: min
  freq: 1000
  strict: false
profiler:
  profile: false
  start_step: 0
  end_step: 100
  name: nvtx
network_dir: outputs
initialization_network_dir: ''
save_filetypes: vtk,npz
summary_histograms: false
jit: true
jit_use_nvfuser: true
jit_arch_mode: only_activation
jit_autograd_nodes: false
cuda_graphs: true
cuda_graph_warmup: 20
find_unused_parameters: false
broadcast_buffers: false
device: ''
debug: fal

### 定义必要组件

#### PDE

本案例特殊之处在于需要自行定义PDE和边界条件

自定义PDE通过继承抽象基类(modulus.sym.eq.pde.PDE)实现

In [4]:
class WaveEquation1D(PDE):
    """
    Wave equation 1D
    The equation is given as an example for implementing
    your own PDE. A more universal implementation of the
    wave equation can be found by
    `from modulus.sym.eq.pdes.wave_equation import WaveEquation`.

    Parameters
    ==========
    c : float, string
        Wave speed coefficient. If a string then the
        wave speed is input into the equation.
    """

    name = "WaveEquation1D"

    def __init__(self, c=1.0):
        # 空间坐标
        x = Symbol("x")

        # 时间坐标
        t = Symbol("t")

        # 输入变量
        input_variables = {"x": x, "t": t}

        # 输出变量
        u = Function("u")(*input_variables)

        # 若传入的c为字符串，则表明c是随输入变量变化的量（inverse）
        # 否则定义为Number
        if type(c) is str:
            c = Function(c)(*input_variables)
        elif type(c) in [float, int]:
            c = Number(c)

        # 构建方程
        self.equations = {}
        self.equations["wave_equation"] = u.diff(t, 2) - (c**2 * u.diff(x)).diff(x)

we = WaveEquation1D(c=1.0)

#### Model

In [5]:
# 输入为空间坐标和时间坐标
# 输出为u
wave_net = instantiate_arch(
        input_keys=[Key("x"), Key("t")],
        output_keys=[Key("u")],
        cfg=cfg.arch.fully_connected,
    )
print(wave_net)

nodes = we.make_nodes() + [wave_net.make_node(name="wave_network")]

FullyConnectedArch(
  (_impl): FullyConnectedArchCore(
    (layers): ModuleList(
      (0): FCLayer(
        (linear): WeightNormLinear(in_features=2, out_features=512, bias=True)
      )
      (1-5): 5 x FCLayer(
        (linear): WeightNormLinear(in_features=512, out_features=512, bias=True)
      )
    )
    (final_layer): FCLayer(
      (activation_fn): Identity()
      (linear): Linear(in_features=512, out_features=1, bias=True)
    )
  )
)


#### Geo

In [6]:
# 定义求解区域
# 本问题中，x范围为[0, pi]，t范围为[0, 2*pi]
x, t_symbol = Symbol("x"), Symbol("t")
L = float(np.pi)
T = 4 * L

# 注意
# 这里使用了序列参数化OrderedParameterization
# 这将导致序列化采样时间t
# 实现代码中的关键部分如下

# for key, value in _sample_ranges(
#             nr_points, self.param_ranges, quasirandom
#         ).items():
#             # sort the samples for the given key
#             if key == self.key:
#                 if sort == "ascending":  # 关键代码，会尝试对采样到的数据进行排序
#                     value = np.sort(value, axis=0)
#                 elif sort == "descending":
#                     value = np.sort(value, axis=0)[::-1]
#                 else:
#                     raise ValueError(
#                         "Sort must be one of None, 'ascending', or 'descending' (got {})".format(
#                             str(sort)
#                         )
#                     )
#             sample_dict[str(key)] = value


geo = Line1D(
        0, L, parameterization=OrderedParameterization({t_symbol: (0, T)}, key=t_symbol)
    )
time_range = {t_symbol: (0, 2 * L)}

In [7]:
# 测试采样
interior_samples = geo.sample_interior(5)

print(interior_samples)

{'x': array([[3.08081287],
       [1.66150172],
       [0.09600378],
       [2.98242516],
       [1.65035974]]), 'sdf': array([[0.06077978],
       [1.48009094],
       [0.09600378],
       [0.15916749],
       [1.49123291]]), 'area': array([[0.62831853],
       [0.62831853],
       [0.62831853],
       [0.62831853],
       [0.62831853]]), 't': array([[ 0.71822547],
       [ 1.53923369],
       [ 2.83266931],
       [ 5.20372526],
       [11.90648326]])}


#### Domain

在Domain中定义约束以及训练所需的各种组件

In [8]:
# make domain
domain = Domain()

#### 边界条件

In [9]:
# boundary condition
BC = PointwiseBoundaryConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"u": 0},
    batch_size=cfg.batch_size.BC,
    lambda_weighting={"u": 100.0},
#     parameterization=time_range,  # 由于在Geo的定义中进行了参数化，因此这里就不用参数化了，否则会覆盖上面的参数化
)
domain.add_constraint(BC, "BC")


初始条件

初始条件需要计算梯度，在Modulus中，梯度的符号表示方法是`__`。例如对$x$的一阶导：`__x`；二阶导：`__x__x`

In [10]:
# initial condition
IC = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"u": sin(x), "u__t": sin(x)},
    batch_size=cfg.batch_size.IC,
    lambda_weighting={"u": 100.0, "u__t": 1.0},
    parameterization={t_symbol: 0.0},  # 这里依然需要指定范围为0.0，从而覆盖上面的参数化结果
)
domain.add_constraint(IC, "IC")

内部满足PDE约束

In [11]:
#     def __init__(self, ord: int = 2, eps: float = 1.0, n_chunks=10):  # 默认10组
#         super().__init__()
#         self.ord: int = ord
#         self.eps: float = eps
#         self.n_chunks: int = n_chunks

#     @staticmethod
#     def _loss(
#         invar: Dict[str, Tensor],
#         pred_outvar: Dict[str, Tensor],
#         true_outvar: Dict[str, Tensor],
#         lambda_weighting: Dict[str, Tensor],
#         step: int,
#         ord: float,
#         eps: float,
#         n_chunks: int,
#     ) -> Dict[str, Tensor]:
#         losses = {}
#         for key, value in pred_outvar.items():
#             l = lambda_weighting[key] * torch.abs(
#                 pred_outvar[key] - true_outvar[key]
#             ).pow(ord)

#             if "area" in invar.keys():
#                 l *= invar["area"]

#             # batch size should be divided by the number of chunks
#             if l.shape[0] % n_chunks != 0:
#                 raise ValueError(
#                     "The batch size must be divided by the number of chunks"
#                 )
#             # divide the loss values into chunks
#             l = l.reshape(n_chunks, -1)
#             l = l.sum(axis=-1)
#             # compute causal temporal weights
#             with torch.no_grad():
#                 w = torch.exp(-eps * torch.cumsum(l, dim=0))  # 论文中的实现，和兴部分
#                 w = w / w[0]

#             l = w * l
#             losses[key] = l.sum()

#         return losses

In [12]:
# interior
interior = PointwiseInteriorConstraint(
    nodes=nodes,
    geometry=geo,
    outvar={"wave_equation": 0},
    batch_size=cfg.batch_size.interior,
    loss=CausalLossNorm(eps=1.0),  # 引入CausalLoss
    fixed_dataset=False,  # 允许在空间中重复采样
    shuffle=False,  # 取消shuffle，保证序列
)
domain.add_constraint(interior, "interior")

验证器以及其他必要组件

In [13]:
# x和t的step size
deltaT = 0.01
deltaX = 0.01

# 创建x和t序列
x = np.arange(0, L, deltaX)
t = np.arange(0, 2 * L, deltaT)

# 构建网格
X, T = np.meshgrid(x, t)
X = np.expand_dims(X.flatten(), axis=-1)
T = np.expand_dims(T.flatten(), axis=-1)

# 解析解
u = np.sin(X) * (np.cos(T) + np.sin(T))

# 创建输入输出
invar_numpy = {"x": X, "t": T}
outvar_numpy = {"u": u}

# 创建Validator
validator = PointwiseValidator(
    nodes=nodes, invar=invar_numpy, true_outvar=outvar_numpy, batch_size=128, plotter=ValidatorPlotter()
)
domain.add_validator(validator)

### 求解器以及求解

In [14]:
# 定义求解器
slv = Solver(cfg, domain)

手动加载日志系统

In [15]:
import logging
# logging.getLogger().addHandler(logging.StreamHandler())
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# create console handler and set level to debug
ch = logging.StreamHandler()
ch.setLevel(logging.INFO)

# create formatter
formatter = logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s')

# add formatter to ch
ch.setFormatter(formatter)

# add ch to logger
logger.addHandler(ch)

启动求解

In [16]:
slv.solve()

2024-02-27 08:18:16,461 - modulus.sym.trainer - WARNING - Installed PyTorch version 2.2.0a0+81ea7a4 is not TorchScript supported in Modulus. Version 2.1.0a0+4136153 is officially supported.
2024-02-27 08:18:16,473 - modulus.sym.trainer - INFO - attempting to restore from: /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:18:16,474 - modulus.sym.trainer - WARNING - optimizer checkpoint not found
2024-02-27 08:18:16,474 - modulus.sym.trainer - WARNING - model wave_network.0.pth not found
2024-02-27 08:18:18,257 - modulus.sym.trainer - INFO - [step:          0] record constraint batch time:  5.491e-02s
2024-02-27 08:18:28,863 - modulus.sym.trainer - INFO - [step:          0] record validators time:  1.060e+01s
2024-02-27 08:18:28,919 - modulus.sym.trainer - INFO - [step:          0] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:18:28,920 - modulus.sym.trainer - INFO - [step:          0] loss:  3.292e+02
2024-02-27 08:18:30,675 - modulus.sym.trainer - INFO - Attem

2024-02-27 08:20:48,076 - modulus.sym.trainer - INFO - [step:       4800] loss:  6.291e-02, time/iteration:  1.732e+01 ms
2024-02-27 08:20:49,801 - modulus.sym.trainer - INFO - [step:       4900] loss:  8.805e-02, time/iteration:  1.723e+01 ms
2024-02-27 08:20:51,764 - modulus.sym.trainer - INFO - [step:       5000] record constraint batch time:  9.238e-02s
2024-02-27 08:21:02,065 - modulus.sym.trainer - INFO - [step:       5000] record validators time:  1.030e+01s
2024-02-27 08:21:02,143 - modulus.sym.trainer - INFO - [step:       5000] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:21:02,146 - modulus.sym.trainer - INFO - [step:       5000] loss:  1.136e-01, time/iteration:  1.234e+02 ms
2024-02-27 08:21:03,921 - modulus.sym.trainer - INFO - [step:       5100] loss:  5.338e-02, time/iteration:  1.774e+01 ms
2024-02-27 08:21:05,680 - modulus.sym.trainer - INFO - [step:       5200] loss:  6.167e-02, time/iteration:  1.758e+01 ms
2024-02-27 08:21:07,439 - modulus

2024-02-27 08:23:25,150 - modulus.sym.trainer - INFO - [step:      10000] record validators time:  1.101e+01s
2024-02-27 08:23:25,189 - modulus.sym.trainer - INFO - [step:      10000] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:23:25,190 - modulus.sym.trainer - INFO - [step:      10000] loss:  6.762e-02, time/iteration:  1.307e+02 ms
2024-02-27 08:23:26,942 - modulus.sym.trainer - INFO - [step:      10100] loss:  6.730e-02, time/iteration:  1.751e+01 ms
2024-02-27 08:23:28,670 - modulus.sym.trainer - INFO - [step:      10200] loss:  6.354e-02, time/iteration:  1.726e+01 ms
2024-02-27 08:23:30,414 - modulus.sym.trainer - INFO - [step:      10300] loss:  5.922e-02, time/iteration:  1.743e+01 ms
2024-02-27 08:23:32,146 - modulus.sym.trainer - INFO - [step:      10400] loss:  8.678e-02, time/iteration:  1.731e+01 ms
2024-02-27 08:23:33,983 - modulus.sym.trainer - INFO - [step:      10500] loss:  6.949e-02, time/iteration:  1.834e+01 ms
2024-02-27 08:23:35,814 - m

2024-02-27 08:25:48,201 - modulus.sym.trainer - INFO - [step:      15100] loss:  6.464e-02, time/iteration:  1.734e+01 ms
2024-02-27 08:25:49,925 - modulus.sym.trainer - INFO - [step:      15200] loss:  2.843e-02, time/iteration:  1.722e+01 ms
2024-02-27 08:25:51,653 - modulus.sym.trainer - INFO - [step:      15300] loss:  4.728e-02, time/iteration:  1.726e+01 ms
2024-02-27 08:25:53,375 - modulus.sym.trainer - INFO - [step:      15400] loss:  6.862e-02, time/iteration:  1.721e+01 ms
2024-02-27 08:25:55,101 - modulus.sym.trainer - INFO - [step:      15500] loss:  2.834e-02, time/iteration:  1.724e+01 ms
2024-02-27 08:25:56,822 - modulus.sym.trainer - INFO - [step:      15600] loss:  2.413e-02, time/iteration:  1.719e+01 ms
2024-02-27 08:25:58,552 - modulus.sym.trainer - INFO - [step:      15700] loss:  2.802e-02, time/iteration:  1.728e+01 ms
2024-02-27 08:26:00,276 - modulus.sym.trainer - INFO - [step:      15800] loss:  2.399e-02, time/iteration:  1.722e+01 ms
2024-02-27 08:26:02,003 

2024-02-27 08:28:16,343 - modulus.sym.trainer - INFO - [step:      20400] loss:  2.546e-03, time/iteration:  1.724e+01 ms
2024-02-27 08:28:18,109 - modulus.sym.trainer - INFO - [step:      20500] loss:  5.683e-03, time/iteration:  1.765e+01 ms
2024-02-27 08:28:19,904 - modulus.sym.trainer - INFO - [step:      20600] loss:  1.913e-03, time/iteration:  1.793e+01 ms
2024-02-27 08:28:21,729 - modulus.sym.trainer - INFO - [step:      20700] loss:  2.955e-03, time/iteration:  1.824e+01 ms
2024-02-27 08:28:23,521 - modulus.sym.trainer - INFO - [step:      20800] loss:  8.836e-03, time/iteration:  1.791e+01 ms
2024-02-27 08:28:25,279 - modulus.sym.trainer - INFO - [step:      20900] loss:  1.381e-03, time/iteration:  1.756e+01 ms
2024-02-27 08:28:27,271 - modulus.sym.trainer - INFO - [step:      21000] record constraint batch time:  9.623e-02s
2024-02-27 08:28:38,243 - modulus.sym.trainer - INFO - [step:      21000] record validators time:  1.097e+01s
2024-02-27 08:28:38,337 - modulus.sym.trai

...

2024-02-27 08:45:59,055 - modulus.sym.trainer - INFO - [step:      57000] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:45:59,057 - modulus.sym.trainer - INFO - [step:      57000] loss:  1.127e-04, time/iteration:  1.435e+02 ms
2024-02-27 08:46:00,800 - modulus.sym.trainer - INFO - [step:      57100] loss:  2.277e-04, time/iteration:  1.741e+01 ms
2024-02-27 08:46:02,530 - modulus.sym.trainer - INFO - [step:      57200] loss:  1.461e-04, time/iteration:  1.728e+01 ms
2024-02-27 08:46:04,268 - modulus.sym.trainer - INFO - [step:      57300] loss:  1.315e-04, time/iteration:  1.736e+01 ms
2024-02-27 08:46:06,033 - modulus.sym.trainer - INFO - [step:      57400] loss:  1.149e-04, time/iteration:  1.764e+01 ms
2024-02-27 08:46:07,804 - modulus.sym.trainer - INFO - [step:      57500] loss:  5.374e-04, time/iteration:  1.769e+01 ms
2024-02-27 08:46:09,590 - modulus.sym.trainer - INFO - [step:      57600] loss:  1.319e-04, time/iteration:  1.785e+01 ms
2024-02-27 08:4

2024-02-27 08:48:28,273 - modulus.sym.trainer - INFO - [step:      62200] loss:  4.181e-05, time/iteration:  1.721e+01 ms
2024-02-27 08:48:30,022 - modulus.sym.trainer - INFO - [step:      62300] loss:  5.596e-05, time/iteration:  1.747e+01 ms
2024-02-27 08:48:31,783 - modulus.sym.trainer - INFO - [step:      62400] loss:  1.710e-04, time/iteration:  1.760e+01 ms
2024-02-27 08:48:33,604 - modulus.sym.trainer - INFO - [step:      62500] loss:  5.950e-05, time/iteration:  1.819e+01 ms
2024-02-27 08:48:35,423 - modulus.sym.trainer - INFO - [step:      62600] loss:  1.658e-04, time/iteration:  1.818e+01 ms
2024-02-27 08:48:37,226 - modulus.sym.trainer - INFO - [step:      62700] loss:  4.981e-05, time/iteration:  1.802e+01 ms
2024-02-27 08:48:38,988 - modulus.sym.trainer - INFO - [step:      62800] loss:  9.872e-05, time/iteration:  1.760e+01 ms
2024-02-27 08:48:40,759 - modulus.sym.trainer - INFO - [step:      62900] loss:  4.757e-05, time/iteration:  1.769e+01 ms
2024-02-27 08:48:42,829 

2024-02-27 08:50:57,355 - modulus.sym.trainer - INFO - [step:      67500] loss:  9.297e-05, time/iteration:  1.724e+01 ms
2024-02-27 08:50:59,072 - modulus.sym.trainer - INFO - [step:      67600] loss:  5.491e-05, time/iteration:  1.716e+01 ms
2024-02-27 08:51:00,815 - modulus.sym.trainer - INFO - [step:      67700] loss:  5.217e-05, time/iteration:  1.741e+01 ms
2024-02-27 08:51:02,553 - modulus.sym.trainer - INFO - [step:      67800] loss:  7.059e-05, time/iteration:  1.737e+01 ms
2024-02-27 08:51:04,332 - modulus.sym.trainer - INFO - [step:      67900] loss:  1.505e-04, time/iteration:  1.772e+01 ms
2024-02-27 08:51:06,415 - modulus.sym.trainer - INFO - [step:      68000] record constraint batch time:  8.511e-02s
2024-02-27 08:51:17,137 - modulus.sym.trainer - INFO - [step:      68000] record validators time:  1.072e+01s
2024-02-27 08:51:17,184 - modulus.sym.trainer - INFO - [step:      68000] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:51:17,186 - modulus

2024-02-27 08:53:25,745 - modulus.sym.trainer - INFO - [step:      72800] loss:  5.526e-05, time/iteration:  1.845e+01 ms
2024-02-27 08:53:27,564 - modulus.sym.trainer - INFO - [step:      72900] loss:  5.281e-05, time/iteration:  1.818e+01 ms
2024-02-27 08:53:29,605 - modulus.sym.trainer - INFO - [step:      73000] record constraint batch time:  8.443e-02s
2024-02-27 08:53:40,331 - modulus.sym.trainer - INFO - [step:      73000] record validators time:  1.072e+01s
2024-02-27 08:53:40,409 - modulus.sym.trainer - INFO - [step:      73000] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:53:40,411 - modulus.sym.trainer - INFO - [step:      73000] loss:  4.848e-05, time/iteration:  1.285e+02 ms
2024-02-27 08:53:42,276 - modulus.sym.trainer - INFO - [step:      73100] loss:  4.028e-05, time/iteration:  1.863e+01 ms
2024-02-27 08:53:44,098 - modulus.sym.trainer - INFO - [step:      73200] loss:  4.146e-05, time/iteration:  1.821e+01 ms
2024-02-27 08:53:45,925 - modulus

2024-02-27 08:56:00,808 - modulus.sym.trainer - INFO - [step:      78000] record validators time:  9.553e+00s
2024-02-27 08:56:00,926 - modulus.sym.trainer - INFO - [step:      78000] saved checkpoint to /workspace/08_1D_Wave_Causal/outputs
2024-02-27 08:56:00,938 - modulus.sym.trainer - INFO - [step:      78000] loss:  3.672e-05, time/iteration:  1.159e+02 ms
2024-02-27 08:56:02,644 - modulus.sym.trainer - INFO - [step:      78100] loss:  3.455e-05, time/iteration:  1.705e+01 ms
2024-02-27 08:56:04,341 - modulus.sym.trainer - INFO - [step:      78200] loss:  9.565e-05, time/iteration:  1.696e+01 ms
2024-02-27 08:56:06,040 - modulus.sym.trainer - INFO - [step:      78300] loss:  3.688e-05, time/iteration:  1.698e+01 ms
2024-02-27 08:56:07,737 - modulus.sym.trainer - INFO - [step:      78400] loss:  4.811e-05, time/iteration:  1.696e+01 ms
2024-02-27 08:56:09,435 - modulus.sym.trainer - INFO - [step:      78500] loss:  6.132e-05, time/iteration:  1.697e+01 ms
2024-02-27 08:56:11,133 - m

### 后处理以及可视化

对于jupyter，比较方便的方法是使用matplotlib

此外，还可以使用tensorboard以及Paraview

如果使用了PointwiseValidator则可以直接查看验证的结果：

![u](./outputs/validators/validator_u.png)